Output composites conditioned on TREFHT for both CESM and fluxnet stations

In [6]:
import importlib
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os

from glob import glob

from CASutils import filter_utils as filt
from CASutils import calendar_utils as cal

importlib.reload(filt)
importlib.reload(cal)

<module 'CASutils.calendar_utils' from '/home/islas/python/CASanalysis/CASutils/calendar_utils.py'>

In [7]:
datadir="/project/cas/islas/python_savs/fluxnet/"
datadircesm="/project/cas/islas/python_savs/snowpaper/DATA_SORT/fluxnetlocs/"
outfile="/project/cas/islas/python_savs/snowpaper/DATA_SORT/trefhtptile_composites/fluxnetcomposites.nc"

In [8]:
# deseasonalize by filtering out the first 4 harmonics of the seasonal cycle and then removing the seasonal mean of each year
def calcdeseas(da):
    datseas = da.groupby('time.dayofyear').mean('time')
    dat4harm = filt.calc_season_nharm(datseas, 4, dimtime=0)
    anoms = da.groupby('time.dayofyear') - dat4harm
    datdeseas = cal.group_season_daily(anoms, 'DJF')
    seasmean = datdeseas.mean('day')
    datdeseas = datdeseas - seasmean
    datdeseas = np.array(datdeseas).flatten()
    return datdeseas

In [11]:
# sort out the fluxnet data.  Only use the data with quality control flag > 0.5 (!! not sure how appropriate this is!!)
def sortfnetdata(fnetdata):
    data = xr.open_dataset(fnetdata)
    # get rid of 29th Feb on leap years
    data = data.sel(time=~((data.time.dt.month == 2) & (data.time.dt.day == 29)))
    
    # quality control
    shflx = data.shflx.where(data.shflx_qc > 0.5)
    tas = data.tas.where(data.tas_qc > 0.5)
    g = data.g.where( (data.g_qc > 0.5) & (data.g > -999))
    netrad = data.netrad.where( (data.netrad_qc > 0.5))
    
    daystr = xr.DataArray(data.indexes['time'].strftime('%m-%d'), coords = data.time.coords, name='daystr')
    
    shflxseason = shflx.groupby(daystr).mean('time', skipna=True)
    shflx4harm = filt.calc_season_nharm(shflxseason, 4, dimtime=0)
    shflxanoms = shflx.groupby(daystr) - shflx4harm

    tasseason = tas.groupby(daystr).mean('time', skipna=True)
    tas4harm = filt.calc_season_nharm(tasseason, 4, dimtime=0)
    tasanoms = tas.groupby(daystr) - tas4harm
    
#    gseason = g.groupby(daystr).mean('time', skipna=True)
#    g4harm = filt.calc_season_nharm(gseason, 4, dimtime=0)
#    ganoms = g.groupby(daystr) - g4harm

    netradseason = netrad.groupby(daystr).mean('time', skipna=True)
    netrad4harm = filt.calc_season_nharm(netradseason, 4, dimtime=0)
    netradanoms = netrad.groupby(daystr) - netrad4harm

    djfshflx = cal.group_season_daily(shflxanoms, 'DJF')
    djfshflxm = djfshflx.mean('day', skipna=True)
    djfshflx = djfshflx - djfshflxm

    djftas = cal.group_season_daily(tasanoms, 'DJF')
    djftasm = djftas.mean('day', skipna=True)
    djftas = djftas - djftasm
    
#    djfg = cal.group_season_daily(ganoms, 'DJF')
#    djfgm = djfg.mean('day', skipna=True)
#    djfg = djfg - djfgm
    
    djfnetrad = cal.group_season_daily(netradanoms, 'DJF')
    djfnetradm = djfnetrad.mean('day', skipna=True)
    djfnetrad = djfnetrad - djfnetradm    

    nyears = djftas.year.size   

    djftas = np.array(djftas).flatten()
    djfshflx = np.array(djfshflx).flatten()
#    djfg = np.array(djfg).flatten()
    djfnetrad = np.array(djfnetrad).flatten()
    
    return djftas, djfshflx, djfnetrad, nyears   

In [14]:
# read in the stations
stations = [os.path.basename(x) for x in glob(datadir+'/*')]
count=0
for istation in stations:
    name  = istation[0:6]
    
    # read fluxnet data
    fname = datadir+'/'+istation
    fnet_trefht, fnet_shflx, fnet_netrad, nyears = sortfnetdata(fname)
    
    # read in the CESM Data
    fname = datadircesm+'/'+istation
    cesmdat = xr.open_dataset(fname)
    
    clm5_trefht = calcdeseas(cesmdat.clm5_trefht)
    clm5_shflx = calcdeseas(cesmdat.clm5_shflx)
    clm5_fgr = calcdeseas(cesmdat.clm5_fgr)
    clm5_flns = calcdeseas(cesmdat.clm5_flns)
    clm5_fsns = calcdeseas(cesmdat.clm5_fsns)
    clm5_lhflx = calcdeseas(cesmdat.clm5_lhflx)
    netrad = cesmdat.clm5_flns - cesmdat.clm5_fsns
    clm5_netrad = calcdeseas(netrad)
        
    snowd_trefht = calcdeseas(cesmdat.snowd_trefht)
    snowd_shflx = calcdeseas(cesmdat.snowd_shflx)
    snowd_fgr = calcdeseas(cesmdat.snowd_fgr)
    snowd_flns = calcdeseas(cesmdat.snowd_flns)
    snowd_fsns = calcdeseas(cesmdat.snowd_fsns)
    snowd_lhflx = calcdeseas(cesmdat.snowd_lhflx)
    netrad = cesmdat.snowd_flns - cesmdat.snowd_fsns
    snowd_netrad = calcdeseas(netrad)
    
    # calculate ptile bin ranges
    nblocks=10
    binmin = np.empty([nblocks]) ; binmax = np.empty([nblocks])
    for iblock in np.arange(0,nblocks,1):
        binmin[iblock] = np.percentile(clm5_trefht,iblock*10)
        binmax[iblock] = np.percentile(clm5_trefht,iblock*10+10)
        if (iblock == 0):
            binmin[iblock] = np.percentile(clm5_trefht,1)
        if (iblock == (nblocks-1)):
            binmax[iblock] = np.percentile(clm5_trefht,99)
    
    if (count == 0):
        nyearsout = np.zeros([len(stations)])
        stationnameout=[]
        lonout = np.zeros([len(stations)])
        latout = np.zeros([len(stations)])
        trefhtcomp_clm5 = np.zeros([nblocks,len(stations)])
        shflxcomp_clm5 = np.zeros([nblocks,len(stations)])
        fgrcomp_clm5 = np.zeros([nblocks, len(stations)])
        netradcomp_clm5 = np.zeros([nblocks, len(stations)])
        trefhtcomp_snowd = np.zeros([nblocks,len(stations)])
        shflxcomp_snowd = np.zeros([nblocks,len(stations)])
        fgrcomp_snowd = np.zeros([nblocks, len(stations)])
        netradcomp_snowd = np.zeros([nblocks, len(stations)])
        trefhtcomp_fnet = np.zeros([nblocks,len(stations)])
        shflxcomp_fnet = np.zeros([nblocks,len(stations)])
        netradcomp_fnet = np.zeros([nblocks, len(stations)])
        
    for iblock in np.arange(0,nblocks,1):
        
        trefhtcomp_clm5[iblock,count] = (clm5_trefht[ (clm5_trefht >= binmin[iblock]) & (clm5_trefht <binmax[iblock])]).mean()
        shflxcomp_clm5[iblock,count] = (clm5_shflx[ (clm5_trefht >= binmin[iblock]) & (clm5_trefht < binmax[iblock])]).mean()
        fgrcomp_clm5[iblock,count] = (clm5_fgr[ (clm5_trefht >= binmin[iblock]) & (clm5_trefht < binmax[iblock])]).mean()
        netradcomp_clm5[iblock,count] = (clm5_netrad[(clm5_trefht >= binmin[iblock]) & (clm5_trefht < binmax[iblock])]).mean()
        
        trefhtcomp_snowd[iblock,count] = (snowd_trefht[ (snowd_trefht >= binmin[iblock]) & (snowd_trefht <binmax[iblock])]).mean()
        shflxcomp_snowd[iblock,count] = (snowd_shflx[ (snowd_trefht >= binmin[iblock]) & (snowd_trefht < binmax[iblock])]).mean()
        fgrcomp_snowd[iblock,count] = (snowd_fgr[ (snowd_trefht >= binmin[iblock]) & (snowd_trefht < binmax[iblock])]).mean()
        netradcomp_snowd[iblock,count] = (snowd_netrad[(snowd_trefht >= binmin[iblock]) & (snowd_trefht < binmax[iblock])]).mean()
        
        trefhtcomp_fnet[iblock,count] = np.nanmean(fnet_trefht[ (fnet_trefht >= binmin[iblock]) & (fnet_trefht < binmax[iblock])])
        shflxcomp_fnet[iblock,count] = np.nanmean(fnet_shflx[ (fnet_trefht >= binmin[iblock]) & (fnet_trefht < binmax[iblock])]) 
        netradcomp_fnet[iblock,count] = np.nanmean(fnet_netrad[ (fnet_trefht >= binmin[iblock]) & (fnet_trefht < binmax[iblock])])
        
    stationnameout.append(name)
    lonout[count] = cesmdat.lon.values
    latout[count] = cesmdat.lat.values
    nyearsout[count] = nyears
        
    count=count+1

nyears=11.0
nyears=11.0
nyears=11.0


/project/cas/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/xarray/conventions.py:500: SerializationWarning: variable 'lat' has multiple fill values {1e+36, -900.0}, decoding all values to NaN.
  decode_timedelta=decode_timedelta,
/project/cas/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/xarray/conventions.py:500: SerializationWarning: variable 'lon' has multiple fill values {1e+36, -900.0}, decoding all values to NaN.
  decode_timedelta=decode_timedelta,


nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0


/project/cas/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/ipykernel_launcher.py:73: RuntimeWarning: invalid value encountered in greater_equal
/project/cas/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/ipykernel_launcher.py:73: RuntimeWarning: invalid value encountered in less
/project/cas/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/ipykernel_launcher.py:74: RuntimeWarning: invalid value encountered in greater_equal
/project/cas/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/ipykernel_launcher.py:74: RuntimeWarning: invalid value encountered in less
/project/cas/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/ipykernel_launcher.py:75: RuntimeWarning: invalid value encountered in greater_equal
/project/cas/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/ipykernel_launcher.py:75: RuntimeWarning: invalid value encountered in less


nyears=14.0
nyears=14.0
nyears=14.0


...

/project/cas/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/xarray/conventions.py:500: SerializationWarning: variable 'lat' has multiple fill values {1e+36, -900.0}, decoding all values to NaN.
  decode_timedelta=decode_timedelta,
/project/cas/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/xarray/conventions.py:500: SerializationWarning: variable 'lon' has multiple fill values {1e+36, -900.0}, decoding all values to NaN.
  decode_timedelta=decode_timedelta,


nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=15.0
nyears=15.0
nyears=15.0


/project/cas/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/xarray/conventions.py:500: SerializationWarning: variable 'lat' has multiple fill values {1e+36, -900.0}, decoding all values to NaN.
  decode_timedelta=decode_timedelta,
/project/cas/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/xarray/conventions.py:500: SerializationWarning: variable 'lon' has multiple fill values {1e+36, -900.0}, decoding all values to NaN.
  decode_timedelta=decode_timedelta,


nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0
nyears=35.0


/project/cas/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/ipykernel_launcher.py:73: RuntimeWarning: invalid value encountered in greater_equal
/project/cas/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/ipykernel_launcher.py:73: RuntimeWarning: invalid value encountered in less
/project/cas/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/ipykernel_launcher.py:74: RuntimeWarning: invalid value encountered in greater_equal
/project/cas/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/ipykernel_launcher.py:74: RuntimeWarning: invalid value encountered in less
/project/cas/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/ipykernel_launcher.py:75: RuntimeWarning: invalid value encountered in greater_equal
/project/cas/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/ipykernel_launcher.py:75: RuntimeWarning: invalid value encountered in less


In [13]:
print(nyears)

11


In [15]:
print(fnet_trefht.shape)

(990,)


In [16]:
ptile = np.arange(0,10,1)+0.5
trefhtcomp_clm5_xr = xr.DataArray(trefhtcomp_clm5, name='trefhtcomp_clm5', coords=[ptile, stationnameout], dims=['ptile','station'])
shflxcomp_clm5_xr = xr.DataArray(shflxcomp_clm5, name='shflxcomp_clm5', coords = [ptile, stationnameout], dims=['ptile','station'])
fgrcomp_clm5_xr = xr.DataArray(fgrcomp_clm5, name='fgrcomp_clm5', coords=[ptile, stationnameout], dims=['ptile','station'])
netradcomp_clm5_xr = xr.DataArray(netradcomp_clm5, name='netradcomp_clm5', coords=[ptile, stationnameout], dims=['ptile','station'])
trefhtcomp_snowd_xr = xr.DataArray(trefhtcomp_snowd, name='trefhtcomp_snowd', coords=[ptile, stationnameout], dims=['ptile','station'])
shflxcomp_snowd_xr = xr.DataArray(shflxcomp_snowd, name='shflxcomp_snowd', coords=[ptile, stationnameout], dims=['ptile','station'])
fgrcomp_snowd_xr = xr.DataArray(fgrcomp_snowd, name='fgrcomp_snowd', coords=[ptile, stationnameout], dims=['ptile','station'])
netradcomp_snowd_xr = xr.DataArray(netradcomp_snowd, name='netradcomp_snowd', coords=[ptile, stationnameout], dims=['ptile','station'])
trefhtcomp_fnet_xr = xr.DataArray(trefhtcomp_fnet, name='trefhtcomp_fnet', coords=[ptile, stationnameout], dims=['ptile','station'])
shflxcomp_fnet_xr = xr.DataArray(shflxcomp_fnet, name='shflxcomp_fnet', coords=[ptile, stationnameout], dims=['ptile','station'])
netradcomp_fnet_xr = xr.DataArray(netradcomp_fnet, name='netradcomp_fnet', coords=[ptile, stationnameout], dims=['ptile','station'])
lon_xr = xr.DataArray(lonout, name='lon', coords=[stationnameout], dims=['station'])
lat_xr = xr.DataArray(latout, name='lat', coords=[stationnameout], dims=['station'])
nyears_xr = xr.DataArray(nyearsout, name='nyears', coords=[stationnameout], dims=['station'])


trefhtcomp_clm5_xr.to_netcdf(path=outfile)
shflxcomp_clm5_xr.to_netcdf(path=outfile,mode="a")
fgrcomp_clm5_xr.to_netcdf(path=outfile,mode="a")
netradcomp_clm5_xr.to_netcdf(path=outfile,mode="a")
trefhtcomp_snowd_xr.to_netcdf(path=outfile,mode="a")
shflxcomp_snowd_xr.to_netcdf(path=outfile,mode="a")
fgrcomp_snowd_xr.to_netcdf(path=outfile,mode="a")
netradcomp_snowd_xr.to_netcdf(path=outfile,mode="a")
trefhtcomp_fnet_xr.to_netcdf(path=outfile,mode="a")
shflxcomp_fnet_xr.to_netcdf(path=outfile,mode="a")
netradcomp_fnet_xr.to_netcdf(path=outfile,mode="a")
lon_xr.to_netcdf(path=outfile, mode="a")
lat_xr.to_netcdf(path=outfile, mode="a")
nyears_xr.to_netcdf(path=outfile, mode="a")